# Document Classification with Naive Bayes - Lab

## Introduction

In this lecture, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

You will be able to:  

* Implement document classification using Naive Bayes
* Understand the need for the Laplacian smoothing correction
* Explain how to code a bag of words representation

## Import the Dataset

To start, import the dataset stored in the text file `SMSSpamCollection`.

In [27]:
#Your code here
import pandas as pd
df=pd.read_csv('SMSSpamCollection',sep='\t',names=['label','text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for Class Imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [28]:
#Your code here
import numpy as np
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [29]:
ham_df=df.loc[df.label=='ham']
ham_indices=np.random.choice(ham_df.index,747)
ham_indices

array([1358, 4380,  683, 5033, 2636,  203, 2142, 3569,  587, 3353, 3416,
       1517,  883, 4201, 2134, 5411, 3691,  617, 3373, 5166, 2819, 1610,
       3323, 4605, 2388,   29,   76, 5291, 3960, 2793,  873, 3404, 4095,
       3292, 4740,   38, 3137,  959, 4511, 4637,  746,  925, 5094, 3769,
       5313, 1403, 3158, 4477, 1186,  392, 3612, 2971, 1273, 4343,  776,
       4158, 1922, 1185, 3580, 1871, 3020,   69,  152, 3843, 5128, 3019,
       5208, 4892, 4358,  391, 5025, 5276,  895, 1955, 3505, 3350, 3162,
       4472, 3809, 3566, 2673, 3693, 5273, 2239, 4843, 2410, 1817,  631,
        269, 4884, 3946, 1883,  519, 1919,  679, 3541, 1465,  933, 3787,
       4529, 4528, 5203,   90, 2643, 1863, 1543, 4363, 4045, 3263, 3140,
        989, 4419, 2913,  997, 3928, 1676, 4049, 3943, 1109, 5367, 1497,
       4560, 1561,  708, 5480, 4509, 1323, 2762, 3482, 2842, 3657, 3749,
       2051, 4421, 2861, 1600, 4757, 2412, 3859, 4523, 1696, 5134, 2520,
        691, 2338, 1844, 1195,   22, 4689, 2303,  6

In [30]:
spam_subset=df.loc[df.label=='spam']
ham_subset=df.iloc[ham_indices]
len(ham_subset)

747

In [31]:
new_df=ham_subset.append(spam_subset)
new_df.head()

,label,text
1358,ham,Good afternoon loverboy ! How goes you day ? A...
4380,ham,How are you. Just checking up on you
683,ham,Is xy going 4 e lunch?
5033,ham,Ok i vl..do u know i got adsense approved..
2636,ham,Tiwary to rcb.battle between bang and kochi.


In [32]:
new_df.label.value_counts()

ham     747
spam    747
Name: label, dtype: int64

## Train - Test Split

Now implement a train test split on your dataset.

In [33]:
from sklearn.model_selection import train_test_split
x=new_df.drop('label',axis=1)
y=new_df.label
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

## Create the Word Frequency Dictionary for Each Class

Create a word frequency dictionary for each class.

In [40]:
train_df=x_train.join(y_train)
test_df=x_test.join(y_test)

class_word_freq = {}
classes = train_df.label.unique()
for class_ in classes:
    temp_df = train_df[train_df.label==class_]
    bag = {}
    for row in temp_df.index:
        doc = str(temp_df['text'][row])
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

## Count the Total Corpus Words
Calculate V, the total number of words in the corpus.

In [55]:
#Your code here
all_the_words=set(list(class_word_freq['spam'].keys())+list(class_word_freq['ham'].keys()))
V=len(all_the_words)
V

6206

## Create a Bag of Words Function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [57]:
#Your code here
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [58]:
#Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = p_classes[class_]
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test Out Your Classifier

Finally, test out your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [61]:
#Your Code here
p_classes = dict(train_df.label.value_counts(normalize=True))
classify_doc(train_df.iloc[0]['text'], class_word_freq, p_classes , V, return_posteriors=True)

[-191.17102408855166, -191.03073238361236]


'ham'

In [68]:
y_hat_train = x_train.text.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.702929
True     0.297071
dtype: float64

## Level-Up

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!